In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from joblib import load

In [14]:
typhi_grna = "./gRNA_sequences/typhi_grna.csv"
typhi_targets = "./gRNA_sequences/typhi_targets.csv"
typhi_records = pd.read_csv(typhi_grna)
typhi_records_targets = pd.read_csv(typhi_targets)
typhi_records['gRNA'] = typhi_records['gRNA'].str.lower()
typhi_records_targets['Sequence'] = typhi_records_targets['Sequence'].str.lower()
typhi_records.gRNA[0],typhi_records.Type[0],typhi_records_targets.Sequence[0],typhi_records_targets.Target[0]


('aaagataccagagcccgaat',
 'sty1607',
 'tttaaaagcatttgcggcagagaaaggacttgacggcgctgaaaaagatgcatttcttaagagtgcgtttgaacacttacatgcactaagtaaggctggtgaaccacttagtcttgaaacccttgttaatgccatttggccccaggctcccgaggagttatctggtaaacttgctgctgaagagttggagttatcggatggcttcgttcctgatggtcgagtcattcgggctctggtatcttttaaaggcaagtctaaatattgggaacttaagtttgaccgggaagggaaaactgaaggctatattgattatgatccggaaacaaatatt',
 'sty1607')

In [15]:
dict_base = {'a':'t','c':'g','t':'a','g':'c'}
dict_base['a']
def getComplement(target):
    comp = []
    for i in range(len(target)):
        comp.append(dict_base[target[-i-1]])
    return "".join(np.array(comp).ravel())
getComplement(typhi_records_targets.Sequence[0])

'aatatttgtttccggatcataatcaatatagccttcagttttcccttcccggtcaaacttaagttcccaatatttagacttgcctttaaaagataccagagcccgaatgactcgaccatcaggaacgaagccatccgataactccaactcttcagcagcaagtttaccagataactcctcgggagcctggggccaaatggcattaacaagggtttcaagactaagtggttcaccagccttacttagtgcatgtaagtgttcaaacgcactcttaagaaatgcatctttttcagcgccgtcaagtcctttctctgccgcaaatgcttttaaa'

In [16]:
#Find best alignment for gRNA against target 
def align(target, grna):
    max = 0
    comptarget = getComplement(target)
    comp = False
    #For every starting position of target
    for k in range(len(target)):
      count = 0
      countcomp = 0
      #For every nucleotide of the gRNA sequence 
      for l in range(min(len(target[k:]),len(grna))):
        if target[k+l] == grna[l]:
          count += 1
        if comptarget[k+l] == grna[l]:
          countcomp += 1
      if count >= max:
        max = count
        position = k
      if countcomp >= max:
        max = countcomp
        position = k
        comp = True
    if position < 20 or position + 40 > len(target):
      return None,0

    if comp:
      return comptarget[position-20:position+40],max

    return target[position-20:position+40],max

In [17]:
for k in range(53):
    selected = False
    for i in range(len(typhi_records_targets)):
        target = typhi_records_targets.Sequence[i]
        if typhi_records.gRNA[k] in target or typhi_records.gRNA[k] in getComplement(target):
            selected = True
    print(selected)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [18]:
targets = []
hamming = []
for k in range(len(typhi_records)):
    target,max = align(typhi_records_targets[typhi_records_targets['Target']==typhi_records.Type[k]]['Sequence'].tolist()[0],typhi_records.gRNA[k])
    
    targets.append(target)
    hamming.append(20-max)

In [19]:
np.array(hamming)

array([ 0,  1,  0,  0, 20,  0, 20,  0, 20,  0,  0,  0,  0,  0, 20, 20,  0,
        0, 20, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0, 20,  0])

In [20]:
#Import features from Random Forest Model
clf = load('Models/grnamodel_ds.joblib')
df = pd.read_pickle("./Outputs/df_feat_ds_doubletree.pkl") 
output_ds = df.sort_values('importance_values', ascending = False).head(80)
output_ds

,features,importance_values
690,Motif: ta,0.084573
625,gc_target,0.051790
629,Motif: aac,0.033910
668,Motif: g,0.030482
647,Motif: c,0.026248
...,...,...
158,"targetalignedaposition4, g",0.004466
159,"targetalignedaposition4, t",0.004428
260,"targetposition42, a",0.004418
79,"grnaposition4, t",0.004377


In [21]:
def one_hot_encode(seq):
    mapping = dict(zip("acgt", range(4)))    
    seq2 = [mapping[i] for i in seq]
    return np.eye(4)[seq2]
def one_hot_encode_duplets(seq):
    mapping = dict(zip(('aa', 'ac', 'ag', 'at', 'ca', 'cc', 'cg', 'ct', 'ga', 'gc', 'gg', 'gt', 'ta', 'tc', 'tg', 'tt'), range(16)))
    seq2 = []
    for i in range(len(seq)-1):
      seq2.append(mapping[seq[i:i+2]])
    return np.eye(16)[seq2]
def calcGC(seq):
  count = 0
  for l in range(len(seq)):
    if seq[l] == 'g' or seq[l] == 'c':
      count += 1
  return count/len(seq)
def countMotifs(seq, motifs):
  counts = []
  for motif in motifs:
    count = 0
    for i in range(len(seq) - len(motif)):
      if seq[i:i+len(motif)] == motif:
        count += 1
    counts.append(count/(len(seq) - len(motif)))
  return counts
bases = ['a','c','g','t']
motifs_to_look_for = []
for i in range(len(bases)):
  motifs_to_look_for.append(bases[i])
  for j in range(len(bases)):
    motifs_to_look_for.append(bases[i]+bases[j])
    for k in range(len(bases)):
      motifs_to_look_for.append(bases[i]+bases[j]+bases[k])
      # for l in range(len(bases)):
      #   motifs_to_look_for.append(bases[i]+bases[j]+bases[k]+bases[l])
motifs_to_look_for.append('aaaa')
motifs_to_look_for.append('cccc')
motifs_to_look_for.append('gggg')
motifs_to_look_for.append('tttt')

In [22]:
X_ds = []
true_typhi_records = []
true_typhi_type = []
true_typhi_hamming = []
true_typhi_target = []
k = 20
for i in range(len(typhi_records)):
  ## 1 is not length 20 so ignore
  if len(typhi_records.gRNA[i])!=20:
    continue
  # if no suitable target, ignore
  if targets[i] == None:
    continue
  else: 
    true_typhi_records.append(typhi_records.gRNA[i])
    true_typhi_type.append(typhi_records.Type[i])
    true_typhi_hamming.append(hamming[i])
    true_typhi_target.append(targets[i])
  t = np.concatenate((one_hot_encode(typhi_records.gRNA[i]), one_hot_encode(typhi_records.gRNA[i]), one_hot_encode(targets[i][:k]), one_hot_encode(targets[i][k+20:]), one_hot_encode_duplets(typhi_records.gRNA[i])
                           , calcGC(typhi_records.gRNA[i]), calcGC(targets[i]), countMotifs(typhi_records.gRNA[i], motifs_to_look_for)), axis = None)
  X_ds.append(t)
X_reduced_ds = np.array(X_ds)[:,output_ds.index]

In [23]:
y_pred = clf.predict(X_reduced_ds)
y_pred

array([1.29229603, 1.37963121, 1.73495837, 1.66504478, 1.2879607 ,
       1.51594548, 1.19207221, 1.36268352, 1.54044934, 1.21961125,
       1.43946419, 1.54872405, 1.78892548, 1.72500881, 1.38159004,
       1.49126679, 1.67010462, 1.28815957, 1.17642948, 1.49223971,
       1.56300844, 1.78297302, 1.57021856, 1.31828236, 1.39418155,
       1.36006218, 1.17593675, 1.02225189, 1.654453  , 1.19363476,
       1.19949779, 1.16649701, 1.64546739, 1.15449236, 1.44772397,
       1.17109125, 1.11300167, 1.27765665, 1.62750132, 1.54710941,
       1.07906581, 1.11640483, 1.2704552 , 0.81882301, 1.76450344,
       1.09660587, 1.25716306])

In [24]:
for i in range(len(y_pred)):
    if true_typhi_hamming[i]!=0:
        y_pred[i] = 0

In [25]:
df = pd.DataFrame({'gRNA':true_typhi_records, 'my version of target':true_typhi_target, 'type':true_typhi_type, 'hamming distance':true_typhi_hamming,'predicted_ds_k':y_pred})
df = df.sort_values('predicted_ds_k', ascending = False)
df = df.reset_index(drop = True)
df.to_csv('./Outputs/typhi_kvalues_prediction.csv')
df

,gRNA,my version of target,type,hamming distance,predicted_ds_k
0,agtgtaatctggcagcttta,ggcaaaaaaatcactttttaagtgtaatctggcagctttaagatgc...,sty2879,0,1.788925
1,tctaagcagagggttgcaag,agctaccaaaataacttttatctaagcagagggttgcaagtattgg...,sty2879,0,1.782973
2,ttacccagttctgtctgaaa,agaagaccacaccttttttattacccagttctgtctgaaatttagt...,sty0307,0,1.764503
3,gacttgcctttaaaagatac,acttaagttcccaatatttagacttgcctttaaaagataccagagc...,sty1607,0,1.734958
4,atatcaacaacggcaaaaaa,ttgactgaaaaaattgtttaatatcaacaacggcaaaaaaatcact...,sty2879,0,1.725009
5,aactgtaagaaacttgtccc,caagtattggagattgaacaaaaccaaaagggaggacatgcttatg...,sty2879,0,1.670105
6,ttaagagtgcgtttgaacac,agttttcccttcccggtcaaacttaagttcccaatatttagacttg...,sty1607,0,1.665045
7,tatgcaggcattggttatga,gtgtgaatgacatactgccccggcactggtcggctcaatcagatcg...,sty0307,0,1.654453
8,aaggccgtcttctcccgata,tttttattacccagttctgtctgaaatttagtcggatcacgccaga...,sty0307,0,1.645467
9,ggtaaacctgcggtctggcg,gcctgtttcaatgctccggcccttccacttgccctcttttatcggg...,sty0307,0,1.627501
